In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import keras.layers as L
from keras import optimizers, losses, metrics, Model
from keras.callbacks import EarlyStopping
from keras.models import Sequential, load_model
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import tensorflow_model_optimization as tfmot
import tensorflow_addons as tfa

C:\Users\Umer\AppData\Roaming\Python\Python39\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\Umer\AppData\Roaming\Python\Python39\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an i

In [13]:
inputPath = r"C:\Rotated Equal Data"
inputPath = r"D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\Equal Data"

commands = np.array(tf.io.gfile.listdir(str(inputPath)))
print(commands)
N_CLASSES = len(commands)
print(N_CLASSES)

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

filesArray = []
for path, subdirs, files in os.walk(inputPath):
    for name in files:
        filesArray.append(os.path.join(path + "\\", name))
print(len(filesArray))



#for i in filesArray:
    #print(i)
    
max_length = 1000
df = []
labels = []
for filename in filesArray:
    data = np.loadtxt(filename, dtype=np.float32)
    
    data = data/2 #Normalise
    data = abs(data)

    if len(data) < max_length:
        # pad the time series with zeros to a length of 1000
        data = np.pad(data, [(0, max_length - len(data)), (0, 0)], mode='constant')
    elif len(data) > max_length:
        # truncate the time series to a length of 1000
        data = data[:max_length, :]
    a = filename.split("\\")[-2]
    if a == "True":
        label = [1,0]
    else:
        label = [0,1]
    labels.append(label)

    data = np.expand_dims(data, axis=-1)
    data = np.expand_dims(data, axis=-1)
    df.append(data)

df_array = np.array(df)
labels_array = np.array(labels)

df_array, labels_array = unison_shuffled_copies(df_array, labels_array)

df_array, labels_array = unison_shuffled_copies(df_array, labels_array)

print(df_array.shape)
print(labels_array.shape)

['False' 'True']
2
1810
(1810, 1000, 1, 1)
(1810, 2)


In [3]:
df_array[1].shape

(1000, 1, 1)

In [8]:
import datetime

# Define the log directory for TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

def lr_schedule(epoch, lr):
    if epoch < 5:
        return lr
    else:
        print(0.001 * tf.math.exp(-0.02*((epoch-5))))
        return 0.001 * tf.math.exp(-0.02*((epoch-5)))

# Define the TensorBoard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

    
total_epochs = 0

model = Sequential()

model.add(L.Input(shape=(1000,1, 1)))

# Block 1
model.add(L.Conv2D(8, (100, 1), activation='relu'))
model.add(L.AveragePooling2D((2, 1), strides=(2, 1)))

# Block 2
model.add(L.Conv2D(16, (15, 1), activation='relu'))
model.add(L.AveragePooling2D((2, 1)))

# Block 3
model.add(L.Conv2D(16, (5, 1), activation='relu'))
model.add(L.AveragePooling2D((2, 1)))

# Block 4
model.add(L.Conv2D(16, (3, 1), activation='relu'))
model.add(L.AveragePooling2D((2, 1)))

# Fully connected L
model.add(L.Flatten())
model.add(L.Dense(32, activation='relu'))
model.add(L.Dense(2, activation='softmax'))

model.compile(optimizer=tf.optimizers.SGD(learning_rate = 0.001, momentum=0.9),
              loss=losses.CategoricalCrossentropy(),
              metrics=[metrics.CategoricalAccuracy()])

# model.build((235, 4001, 1))
print(model.summary())

new_epochs = 3
model.fit(df_array, labels_array, shuffle=1,  initial_epoch=total_epochs, epochs=total_epochs+new_epochs, validation_split=0.25, batch_size = 2, callbacks=[tensorboard_callback,lr_scheduler])
total_epochs += new_epochs

model.compile(optimizer=tf.optimizers.Adam(learning_rate = 0.001),
              loss=losses.CategoricalCrossentropy(),
              metrics=[metrics.CategoricalAccuracy()])

new_epochs = 8
model.fit(df_array, labels_array, shuffle=1,  initial_epoch=total_epochs, epochs=total_epochs+new_epochs, validation_split=0.05, batch_size = 32, callbacks=[tensorboard_callback,lr_scheduler])
total_epochs += new_epochs

model.evaluate(df_array, labels_array, batch_size=1)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 901, 1, 8)         808       
                                                                 
 average_pooling2d_16 (Avera  (None, 450, 1, 8)        0         
 gePooling2D)                                                    
                                                                 
 conv2d_17 (Conv2D)          (None, 436, 1, 16)        1936      
                                                                 
 average_pooling2d_17 (Avera  (None, 218, 1, 16)       0         
 gePooling2D)                                                    
                                                                 
 conv2d_18 (Conv2D)          (None, 214, 1, 16)        1296      
                                                                 
 average_pooling2d_18 (Avera  (None, 107, 1, 16)      

KeyboardInterrupt: 

In [9]:
model.evaluate(df_array, labels_array, batch_size=1)

182810/182810 [==============================] - 163s 890us/step - loss: 0.0039 - categorical_accuracy: 0.9988


[0.003925190772861242, 0.9988184571266174]

In [10]:
tf.keras.models.save_model(model, 'model.h5')

In [11]:
# Load your Keras model
model = tf.keras.models.load_model('model.h5')

# Define the pruning parameters
pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(
    initial_sparsity=0.1,
    final_sparsity=0.99,
    begin_step=0,
    end_step=19
)

# Define the pruning callback
pruning_callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir='/tmp')
]

# Create a pruned model
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(model, pruning_schedule=pruning_schedule)

# Train the pruned model
pruned_model.compile(optimizer=tf.optimizers.Adam(learning_rate= 0.000001),
              loss=losses.CategoricalCrossentropy(),
              metrics=[metrics.CategoricalAccuracy()])
pruned_model.fit(df_array, labels_array, shuffle=1, epochs=20, validation_split=0.25, batch_size=128, callbacks=pruning_callbacks)

# Remove the pruning wrappers from the model
pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

# Save the pruned model
pruned_model.save('pruned_model.h5')

C:\Users\Umer\AppData\Roaming\Python\Python39\site-packages\tensorflow_model_optimization\python\core\sparsity\keras\pruning_wrapper.py:212: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  mask = self.add_variable(
C:\Users\Umer\AppData\Roaming\Python\Python39\site-packages\tensorflow_model_optimization\python\core\sparsity\keras\pruning_wrapper.py:219: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  threshold = self.add_variable(
C:\Users\Umer\AppData\Roaming\Python\Python39\site-packages\tensorflow_model_optimization\python\core\sparsity\keras\pruning_wrapper.py:233: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.pruning_step = self.add_variable(


Epoch 1/20
1072/1072 [==============================] - 57s 51ms/step - loss: 0.0034 - categorical_accuracy: 0.9990 - val_loss: 0.0039 - val_categorical_accuracy: 0.9988
Epoch 2/20
1072/1072 [==============================] - 55s 51ms/step - loss: 0.0029 - categorical_accuracy: 0.9992 - val_loss: 0.0034 - val_categorical_accuracy: 0.9989
Epoch 3/20
1072/1072 [==============================] - 53s 49ms/step - loss: 0.0026 - categorical_accuracy: 0.9993 - val_loss: 0.0031 - val_categorical_accuracy: 0.9991
Epoch 4/20
1072/1072 [==============================] - 53s 49ms/step - loss: 0.0024 - categorical_accuracy: 0.9994 - val_loss: 0.0029 - val_categorical_accuracy: 0.9992
Epoch 5/20
1072/1072 [==============================] - 53s 49ms/step - loss: 0.0022 - categorical_accuracy: 0.9995 - val_loss: 0.0027 - val_categorical_accuracy: 0.9993
Epoch 6/20
1072/1072 [==============================] - 53s 49ms/step - loss: 0.0021 - categorical_accuracy: 0.9995 - val_loss: 0.0026 - val_categoric

In [12]:
# Load your Keras model
pruned_model = tf.keras.models.load_model('pruned_model.h5')
pruned_model.compile(optimizer=tf.optimizers.Adam(learning_rate= 0.0001),
              loss=losses.CategoricalCrossentropy(),
              metrics=[metrics.CategoricalAccuracy()])
#pruned_model.evaluate(df_array, labels_array, batch_size=1)

In [98]:
######### NON QUANTIZED ############
pruned_model = tf.keras.models.load_model("pruned_model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [14]:
######### QUANTIZED ############
pruned_model = tf.keras.models.load_model("pruned_model.h5")

def representative_data_gen():
    for input_value in df_array:
        input_value = np.expand_dims(input_value, axis=0)
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT, tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.representative_dataset = representative_data_gen
#converter.target_spec.supported_ops = [tf.float16]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model = converter.convert()

with open('model_quantized.tflite', 'wb') as f:
    f.write(tflite_model)


In [15]:
#interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter = tf.lite.Interpreter(model_path="model_quantized.tflite")
interpreter.allocate_tensors()

In [17]:
# Get the input tensor details
input_details = interpreter.get_input_details()
input_tensor_index = input_details[0]['index']
output_details = interpreter.get_output_details()


# Get the scale and zero-point values from the input tensor
input_tensor_scale = input_details[0]['quantization_parameters']['scales']
input_tensor_zero_point = input_details[0]['quantization_parameters']['zero_points']

output_tensor_scale = output_details[0]['quantization'][0]
output_tensor_zero_point = output_details[0]['quantization'][1]

print("Input tensor scale:", input_tensor_scale)
print("Input tensor zero-point:", input_tensor_zero_point)

print("Output tensor scale:", output_tensor_scale)
print("Output tensor zero-point:", output_tensor_zero_point)

Input tensor scale: [0.00392157]
Input tensor zero-point: [0]
Output tensor scale: 0.00390625
Output tensor zero-point: 0


In [18]:
################## For Quantized only ###################
for j in range(len(df_array)):
    for i in range(len(df_array[0])):
        df_array[j][i][0] = int(df_array[j][i][0] / input_tensor_scale) + input_tensor_zero_point

In [19]:
df_array[21]

array([[[131.]],

       [[131.]],

       [[131.]],

       [[131.]],

       [[131.]],

       [[131.]],

       [[131.]],

       [[131.]],

       [[130.]],

       [[130.]],

       [[130.]],

       [[130.]],

       [[130.]],

       [[132.]],

       [[132.]],

       [[132.]],

       [[132.]],

       [[131.]],

       [[132.]],

       [[132.]],

       [[132.]],

       [[132.]],

       [[130.]],

       [[130.]],

       [[130.]],

       [[131.]],

       [[131.]],

       [[132.]],

       [[132.]],

       [[131.]],

       [[130.]],

       [[130.]],

       [[131.]],

       [[131.]],

       [[131.]],

       [[130.]],

       [[130.]],

       [[129.]],

       [[129.]],

       [[130.]],

       [[131.]],

       [[131.]],

       [[131.]],

       [[131.]],

       [[131.]],

       [[131.]],

       [[131.]],

       [[131.]],

       [[131.]],

       [[131.]],

       [[131.]],

       [[131.]],

       [[130.]],

       [[130.]],

       [[130.]],

       [[1

In [20]:
correct = 0
total = 0
from scipy.stats import pearsonr


for i in range(len(df_array)):
    interpreter.set_tensor(input_details[0]['index'], df_array[i:i+1].astype(np.uint8))
    interpreter.invoke()
    output = (interpreter.get_tensor(output_details[0]['index']) - output_tensor_zero_point)*output_tensor_scale
    #origional = model.predict(df_array[i:i+1], verbose=0)
        
    print(f'Label: {labels_array[i]}  tflite model prediction: {output}')


Label: [1 0]  tflite model prediction: [[0.99609375 0.00390625]]
Label: [1 0]  tflite model prediction: [[0.99609375 0.        ]]
Label: [0 1]  tflite model prediction: [[0.         0.99609375]]
Label: [1 0]  tflite model prediction: [[0.99609375 0.        ]]
Label: [1 0]  tflite model prediction: [[0.99609375 0.        ]]
Label: [0 1]  tflite model prediction: [[0.         0.99609375]]
Label: [0 1]  tflite model prediction: [[0.         0.99609375]]
Label: [1 0]  tflite model prediction: [[0.99609375 0.        ]]
Label: [1 0]  tflite model prediction: [[0.99609375 0.        ]]
Label: [1 0]  tflite model prediction: [[0.99609375 0.        ]]
Label: [0 1]  tflite model prediction: [[0.         0.99609375]]
Label: [1 0]  tflite model prediction: [[0.99609375 0.        ]]
Label: [1 0]  tflite model prediction: [[0.99609375 0.        ]]
Label: [0 1]  tflite model prediction: [[0.         0.99609375]]
Label: [1 0]  tflite model prediction: [[0.99609375 0.        ]]
Label: [1 0]  tflite mode